# libraries and packages need

In [ ]:
pip install altair

In [ ]:
pip install vega_datasets

In [3]:
# importing libraries
import pandas as pd
import altair as alt
from vega_datasets import data

# Load the datasets

In [5]:
# Load the datasets 
payments_df = pd.read_csv('D:\Swansea University 2023-2024\Visulisation\Datasets\olist_order_payments_dataset.csv')
orders_df = pd.read_csv('D:\Swansea University 2023-2024\Visulisation\Datasets\olist_orders_dataset.csv')
products_df = pd.read_csv('D:\Swansea University 2023-2024\Visulisation\Datasets\olist_products_dataset.csv')
order_items_df = pd.read_csv('D:\Swansea University 2023-2024\Visulisation\Datasets\olist_order_items_dataset.csv') 


# Visulisation heatmap chart by altair

In [6]:
# order_purchase_timestamp to datetime and extract the day number
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_df['purchase_day'] = orders_df['order_purchase_timestamp'].dt.day  # Keep only the day number

# Filter orders for November 2017
november_orders = orders_df[(orders_df['order_purchase_timestamp'] >= '2017-11-01') & 
                            (orders_df['order_purchase_timestamp'] < '2017-12-01')]

# Merge datasets
merged_df = pd.merge(november_orders[['order_id', 'purchase_day']], payments_df[['order_id', 'payment_value']], on='order_id')
merged_df = pd.merge(merged_df, order_items_df[['order_id', 'product_id']], on='order_id')
merged_df = pd.merge(merged_df, products_df[['product_id', 'product_category_name']], on='product_id')

# Aggregate data
daily_sales = merged_df.groupby(['purchase_day', 'product_category_name'], as_index=False)['payment_value'].sum()

# the top two selling product categories
top_categories = daily_sales.groupby('product_category_name')['payment_value'].sum().nlargest(2).index.tolist()

# Filter for top categories
filtered_sales = daily_sales[daily_sales['product_category_name'].isin(top_categories)]

# Sort the DataFrame by purchase_day to maintain order on the x-axis
filtered_sales = filtered_sales.sort_values('purchase_day')


# Calculate maximum sales value for each category
max_sales = filtered_sales.groupby('product_category_name')['payment_value'].transform('max')
filtered_sales['is_max'] = filtered_sales['payment_value'] == max_sales

total_sales_by_category = filtered_sales.groupby('product_category_name')['payment_value'].sum().reset_index()


# text(note under the chart).
intro_phrase = "Total monthly selling : "
category_totals = "  &  ".join([f"({row['product_category_name']}: {row['payment_value']:,.2f} thousands)" for index, row in total_sales_by_category.iterrows()])
note_text = intro_phrase + category_totals

note_df = pd.DataFrame({'text': [note_text]})


# Create the heatmap 
# Create the heatmap with increased legend title font size
heatmap = alt.Chart(filtered_sales).mark_rect().encode(
    x=alt.X('purchase_day:O', title='Day of November', axis=alt.Axis(titleFontSize=18, labelFontSize=15)),
    y=alt.Y('product_category_name:N', title='Product Category', axis=alt.Axis(titleFontSize=18, labelFontSize=15)),
    color=alt.Color('payment_value:Q', title='PaymentValue', scale=alt.Scale(scheme='yellowgreenblue'),
                    legend=alt.Legend(titleFontSize=17, labelFontSize=13)),  # Increase legend title font size here
    tooltip=['purchase_day', 'product_category_name', 'payment_value']
).properties(
    title={
      "text": ['Top Two Selling Categories in November 2017'],
      "subtitle": ['Olist E-commerce in Brazil [Brazilian Currency]'],
      "color": "black",
      "subtitleColor": "black",
      "fontSize": 22,
      "subtitleFontSize": 19
    },
    width=900,
    height=450
)

# Create text labels for each cell
text_labels = alt.Chart(filtered_sales).mark_text(baseline='middle', angle=270).encode(
    x=alt.X('purchase_day:O'),
    y=alt.Y('product_category_name:N'),
    text=alt.Text('payment_value:Q', format=',.0f'),
    # Conditional color: if the value is maximum within the category, set color to white; otherwise, black
    color=alt.condition(
        alt.datum.is_max,  # Condition: if the row's payment_value is the maximum in its category
        alt.value('white'),  # True value: white color
        alt.value('black')   # False value: black color
    )
).properties(
    width=900,
    height=450
)
text_chart = alt.Chart(note_df).mark_text(size=18, align='left').encode(
    text=alt.Text('text:N')
).properties(
    height=20 
)

# Combine the charts
final_chart = alt.layer(heatmap, text_labels).resolve_scale(
    x='shared',
    y='shared'
)
final_chart_with_note = alt.vconcat(final_chart, text_chart, spacing=5)

# Display the concatenated chart
final_chart_with_note.display()





alt.VConcatChart(...)